In [103]:
import pandas as pd

In [104]:
file = "D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\train_E6oV3lV.csv"
test = 'D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\test_tweets_anuFYb8.csv'
tweets = pd.read_csv(file)
test_tweets=pd.read_csv(test)

In [105]:
print(tweets.shape, test_tweets.shape)

(31962, 3) (17197, 2)


In [106]:
tweets[tweets.label==1].count()

id       2242
label    2242
tweet    2242
dtype: int64

In [107]:
tweets1=tweets[tweets.label==1]
tweets=tweets.append(tweets1)
tweets=tweets.append(tweets1)

In [108]:
label=tweets.label

In [109]:
tweets[tweets.label==1].count()

id       6726
label    6726
tweet    6726
dtype: int64

In [110]:
tweets_train_test=tweets.append(test_tweets)

C:\Amrendra\Anaconda\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [111]:
cleaned_tweet = tweets_train_test.tweet.str.replace("[^a-zA-Z]", " ")

In [112]:
cleaned_tweet=cleaned_tweet.apply(lambda x :" ".join(w for w in x.split() if len(w)>2 and w != 'user' ))

In [113]:
cleaned_tweet.count()

53643

In [114]:
from nltk.stem.porter import PorterStemmer
#from nltk.stem.snowball import SnowballStemmer

In [115]:
stem=PorterStemmer()

In [116]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( stem.stem(w) for w in x.split()))

In [117]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( w for w in x.split() if len(w)>2))

In [118]:
cleaned_tweet.count()

53643

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_tweet)
sequences = tokenizer.texts_to_sequences(cleaned_tweet)

In [119]:
length = []
for x in cleaned_tweet:
    length.append(len(x.split()))
max(length)

25

In [120]:
#from keras.preprocessing.sequence import pad_sequences
#word_index=pad_sequences(sequences, maxlen=25, padding = 'pre', truncating='pre')

In [121]:
train = cleaned_tweet[:36446]
test = cleaned_tweet[36446:]

In [122]:
train[0]

'when father dysfunct and selfish drag kid into dysfunct run'

In [123]:
from sklearn.model_selection import train_test_split

In [124]:
label.shape

(36446,)

In [125]:
x_train,x_valid,y_train, y_valid = train_test_split(train,label,test_size=0.2,random_state=3, stratify=label)

In [126]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(29156,)
(29156,)
(7290,)
(7290,)


In [127]:
import tensorflow as tf
import tensorflow_hub as hub
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout,Input, Lambda
from keras.layers import Conv1D, SpatialDropout1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as ply
%matplotlib inline

In [129]:
output_dir = 'model_tweet_output/dense'
epochs = 2
batch_size = 128
n_dim = 64
n_unique_words = 50000
n_words_to_skip = 5
max_review_length = 25
pad_type = trunc_type = 'pre'
drop_embed = 0.2

n_dense = 256
dropout = 0.2

n_conv = 256
k_conv = 3

In [130]:
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)

dense = Dense(64, activation='relu')(embedding)
pred = Dense(1, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [131]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                65600     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 65,665
Trainable params: 65,665
Non-trainable params: 0
_________________________________________________________________


In [132]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [133]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")

In [134]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [135]:
model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid,y_valid), callbacks=[modelcheckpoint])

Train on 29156 samples, validate on 7290 samples
Epoch 1/2
15104/29156 [==============>...............] - ETA: 23:55 - loss: 12.9321 - acc: 0.1888

KeyboardInterrupt: 

In [39]:
model.load_weights(output_dir+'/weights.02.hdf5')

In [40]:
y_hat = model.predict(test)

In [41]:
pd.DataFrame(y_hat).to_csv('submit_nn.csv')

In [ ]:
y_hat